# Process and prepare the raw data for use in the lesson

The raw gravity and topography data are generated from [spherical harmonic models](https://en.wikipedia.org/wiki/Spherical_harmonics) using the [ICGEM Calculation Service](http://icgem.gfz-potsdam.de). The data are distributed in text files and heights are defined relative to the geoid (orthometric heights). We need to convert the obeservation heights and topography to geometric heights (relative to the ellipsoid) using the geoid heights also downloaded from ICGEM. We'll also save this data in [netCDF](https://www.unidata.ucar.edu/software/netcdf/) files for easier loading with [xarray](http://xarray.pydata.org/).

The model used to generate the gravity and geoid height data is EIGEN-6c4. The topography data is interpolated from the ETOPO1 model.

## Import the required libraries

We need the following libraries to do this work.

In [1]:
import numpy as np
import xarray as xr

## Loading the data and converting to an xarray grid

The data are laidout in regular grids but ICGEM makes them available in a text format. That's not very efficient for computation or storage. Plus, for each location we have 3 different files: the raw gravity (with the measurement height), the geoid, and the ETOPO1 topography. 

We can use [`xarray.Dataset`](http://xarray.pydata.org/en/stable/data-structures.html#dataset)s to store all of these grids in a single variable and then export them into a single netCDF file.

We'll start by writing a function that loads the data in a ICGEM `.gdf` file and stores them in an [`xarray.Dataset`](http://xarray.pydata.org/en/stable/data-structures.html#dataset).

In [2]:
def load_icgem_gdf(fname, dtype = 'float32'):
    """
    Load data from an ICGEM .gdf file into an xarray.Dataset.
    
    Reads metdata from the header, like the grid area, number of points, 
    height over the ellipsoid (if it's constant), etc.
    
    Stores the file header in the ``attrs`` attribute of the Dataset.

    Parameters
    ----------
    fname : str
        The name of the .gdf file.
    dtype : str or numpy dtype object
        The data type used when loading the data from the file.

    Returns
    -------
    data : xarray.Dataset
     
    """    
    with open(fname) as gdf_file:
        # Read the header and extract metadata
        header = []
        shape = [None, None]
        size = None
        height_over_ell = None
        fields = None
        is_field_names = False
        west, east, south, north = [None]*4
        for line in gdf_file:
            if line.strip()[:11] == 'end_of_head':
                break
            if not line.strip():
                # The field names will come after a blank line
                is_field_names = True
                continue
            header.append(line)
            parts = line.strip().split()
            if parts[0] == 'height_over_ell':
                height_over_ell = float(parts[1])
            elif parts[0] == 'latitude_parallels':
                nlat = shape[0] = int(parts[1])
            elif parts[0] == 'longitude_parallels':
                nlon = shape[1] = int(parts[1])
            elif parts[0] == 'number_of_gridpoints':
                size = int(parts[1])
            elif parts[0] == 'latlimit_south':
                south = float(parts[1])
            elif parts[0] == 'latlimit_north':
                north = float(parts[1])
            elif parts[0] == 'longlimit_west':
                west = float(parts[1])
            elif parts[0] == 'longlimit_east':
                east = float(parts[1])
            if is_field_names:
                # Skip the first two because they are the coordinate
                # names.
                fields = line.strip().split()[2:]
                is_field_names = False
        # Read the numerical values
        rawdata = np.loadtxt(gdf_file, ndmin=2, unpack=True, dtype=dtype)
        
    # Sanity checks
    assert all(n is not None for n in shape), "Couldn't read shape of grid."
    assert size is not None, "Couldn't read size of grid."
    assert shape[0]*shape[1] == size, \
        "Grid shape '{}' and size '{}' mismatch.".format(shape, size)
    assert fields is not None, "Couldn't read column names."
    assert len(fields) == rawdata.shape[0] - 2, \
        "Number of attributes ({}) and data columns ({}) mismatch".format(
            len(fields), rawdata.shape[0] - 2)
    assert all(i is not None for i in [west, east, south, north]), \
        "Couldn't read the grid area."
                
    if height_over_ell is not None:
        fields.append('height_over_ell')
        rawdata.append(height_over_ell*np.ones(size, dtype=dtype))
        
    # Build the xarray container
    dims = ['latitude', 'longitude']
    latitude = np.linspace(south, north, nlat, dtype=dtype)
    longitude = np.linspace(west, east, nlon, dtype=dtype)
    # Cartopy doesn't like 0-360 longitude
    longitude[longitude > 180] -= 360
    coords = {'latitude': latitude, 'longitude': longitude}
    attrs = {'file_header': ''.join(header)}
    data_vars = {}
    for name, value in zip(fields, rawdata[2:]):
        # Need to invert the data matrices in latitude "[::-1]"
        # because the ICGEM grid varies latitude from N to S
        # instead of S to N.        
        data_vars[name] = xr.DataArray(
            value.reshape(shape)[::-1], coords=coords, dims=dims, name=name,
            attrs=attrs)    
        
    return xr.Dataset(data_vars, attrs=attrs)    

Let's test this in the Hawai'i topography data.

In [3]:
load_icgem_gdf('etopo1-hawaii.gdf')

<xarray.Dataset>
Dimensions:         (latitude: 301, longitude: 301)
Coordinates:
  * latitude        (latitude) float32 13.0 13.05 13.1 13.15 13.2 13.25 13.3 ...
  * longitude       (longitude) float32 -165.0 -164.95 -164.9 -164.85 -164.8 ...
Data variables:
    topography_grd  (latitude, longitude) float32 -3902.0 -2930.0 -3028.0 ...
Attributes:
    file_header:  generating_institute     gfz-potsdam\n     generating_date ...

Perfect! Now we need a function that loads each dataset from a location and combines them into a single `Dataset`.

In [4]:
def load_location_data(location):
    """
    Load gravity, geoid, and topography data for a given location.
    
    Includes the computed ellipsoidal (geometric) heights computed 
    using the given geoid heights.
    
    Parameters
    ----------
    location : str
        Location name, as used in the data files.
    
    Returns
    -------
    data : xarray.Dataset
        Combined dataset with all grids loaded from different files.
    """
    gravity = load_icgem_gdf('eigen-6c4-{}-gravity.gdf'.format(location))
    geoid = load_icgem_gdf('eigen-6c4-{}-geoid.gdf'.format(location))
    topography = load_icgem_gdf('etopo1-{}.gdf'.format(location))
    data = xr.merge([gravity, geoid, topography])
    # Compute ellipsoidal (geometric) heights
    data['h_over_ellipsoid'] = data.h_over_geoid + data.geoid
    data['topography_ell'] = data.topography_grd + data.geoid
    return data

Let's test it to make sure it works:

In [5]:
load_location_data('hawaii')

<xarray.Dataset>
Dimensions:           (latitude: 301, longitude: 301)
Coordinates:
  * latitude          (latitude) float32 13.0 13.05 13.1 13.15 13.2 13.25 ...
  * longitude         (longitude) float32 -165.0 -164.95 -164.9 -164.85 ...
Data variables:
    gravity_earth     (latitude, longitude) float32 978337.0 978356.0 ...
    h_over_geoid      (latitude, longitude) float32 0.0 0.0 0.0 0.0 0.0 0.0 ...
    geoid             (latitude, longitude) float32 12.0905 12.123 12.0423 ...
    topography_grd    (latitude, longitude) float32 -3902.0 -2930.0 -3028.0 ...
    h_over_ellipsoid  (latitude, longitude) float32 12.0905 12.123 12.0423 ...
    topography_ell    (latitude, longitude) float32 -3889.91 -2917.88 ...

It's important to note that `h_over_geoid` and `h_over_ellipsoid` refer to the observation height of the gravity measurements while `topography_grd` and `topography_ell` are topographic and bathimetric heights with respect to the geoid and ellipsoid (respectively).

Now we can load the data from all locations and save them to netCDF.

In [6]:
locations = 'hawaii'.split()
for location in locations:
    load_location_data(location).to_netcdf('{}-gravity.nc'.format(location), mode='w')

Done!